<h1>IDEA: Un algoritmo de clave privada</h1>

<h2> Cifrado de un mensaje, dados los siguientes datos:</h2>
<li> Mensaje="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"</li>
<li> Clave privada (K) => 128 bits ="MARCOSDIEGOSANTI"

<h2>1. Obtención de los valores binarios del mensaje y la clave </h2>

In [271]:
mensaje_claro="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"
#La clave tiene que usar un alfabeto que contenga menos de 128 bits
clave_privada="MARCOSDIEGOSANTI"
#clave_privada="11112222333344445555666677778888"
alf="AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZ.,;:()¿?¡!-0123456789aábcdeéfghiíjklmnñoópqrstuúvwxyz "

def cadena_a_binario(cadena):
    #Conversion a binario de 8 bits
    #cadena_binaria = "".join(format(int(x), '04b') for x in cadena)
    cadena_binaria = "".join(format(alf.index(x), '08b') for x in cadena)
    return cadena_binaria

mensaje_binario = cadena_a_binario(mensaje_claro)
clave_binaria = cadena_a_binario(clave_privada)

#var = 0b01

#aux = clave_binaria[25:]+clave_binaria[:25]
#lista = [int(aux[x:x+4], 2) for x in range(0,128,4)]
#print(lista)
#print(aux)
#print(len(aux))
#print(var<<1)
print(len(clave_binaria))


128


<h2>2. Se divide el mensaje binario en bloques de 64 bits. Cada bloque se divide a su vez en 4 sub-bloques de 16 bits </h2>


In [272]:
def binario_a_bloques(cadena, tam_bloque):
    lista_bloques = [cadena[x:x+tam_bloque] for x in range(0,len(cadena), tam_bloque)]
    len_ultimo_bloque = len(lista_bloques[-1])
    #Relleno con 0s si no se alcanza el tamaño del bloque
    if(len_ultimo_bloque!=tam_bloque):
        lista_bloques[-1] = lista_bloques[-1]+(tam_bloque-len_ultimo_bloque)*"0"
    return lista_bloques

#Mensaje dividido en bloques de 64 bits, que tendrá la forma [ab,cd]
mensaje_dividido = binario_a_bloques(mensaje_binario, 64)

#Cada sub-bloque lo divido en 4 bloques de 16 bits, guardo el resultado final, que tendrá la forma [[a,b],[c,d]]
mensaje_bloques = []
for bloque in mensaje_dividido:
    mensaje_bloques.append(binario_a_bloques(bloque, 16))

print(int(mensaje_bloques[0][0],2))



1349


<h2>3. Se calculan las claves intermedias. Serán un total de 52 subclaves de 16 bits, que se emplearán para cifrar el mensaje</h2>

In [273]:
 #Recibe un string de valores binarios
 def generar_claves_intermedias_cifrado(clave_inicial):
     # lista de una dimensión, que contendrá las 52 subclaves
     lista_claves = []
     #La agrupación será de 6 en 6 subclaves. Las 8 primeras son la clave inicial.
     lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     while(len(lista_claves)<52):
         #Desplazar 25 bits a la izquierda la clave inicial
         clave_inicial = clave_inicial[25:]+clave_inicial[:25]
         #Bloques de la clave actualizada
         lista_claves.extend(binario_a_bloques(clave_inicial, 16))

     # lista_claves crece en múltiplos de 8 y es por ello que debemos quedarnos solo con los 52 primeros valores
     lista_claves = lista_claves[:52]
     #print(lista_claves)
     return lista_claves

claves_intermedias = generar_claves_intermedias_cifrado(clave_binaria)
#print(claves_intermedias)


<h2>4. Aplico el algoritmo de cifrado a cada bloque del mensaje</h2>

In [274]:
def aplicar_cifrado(bloque_msj, claves_intermedias):
    for i in range(0,8):
        #En cada iteración se cogen 6 subclaves
        subclaves = claves_intermedias[i*6:(i*6)+6]
        
        #Paso 1
        var1 = int(bloque_msj[0],2) * int(claves_intermedias[0],2)

        #Paso 2
        var2 = int(bloque_msj[1],2) + int(claves_intermedias[1],2)

        #Paso 3
        var3 = int(bloque_msj[2],2) + int(claves_intermedias[2],2)

        #Paso 4
        var4 = int(bloque_msj[3],2) * int(claves_intermedias[3],2)

        #Paso 5 (XOR entre 1 y 3)
        var5 = var1 ^ var2

        #Paso 6 
        var6 = var2 ^ var4

        #Paso 7
        var7 = var5 * int(claves_intermedias[4], 2)

        #Paso 8
        var8 = var6 + var7

        #Paso 9
        var9 = var8 * int(claves_intermedias[5], 2)

        #Paso 10
        var10 = var7 + var9

        #Paso 11
        var11 = var1 ^ var9

        #Paso 12
        var11 = var3 ^ var9

        #Paso 13
        var11 = var3 ^ var10

        #Paso 14
        var11 = var4 ^ var10



aplicar_cifrado(mensaje_bloques[0], claves_intermedias)